In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from matplotlib import pyplot as plt
import json
import numpy as np
import re
import random
import pandas as pd
from matplotlib import pyplot as plt
import json
import numpy as np
import re
import random
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import pickle
from scipy.sparse import coo_matrix, hstack

from sklearn.metrics.cluster import homogeneity_score
from sklearn.metrics.cluster import v_measure_score
from sklearn.metrics.cluster import completeness_score

from sklearn.decomposition import TruncatedSVD

from sklearn.cluster import KMeans, MiniBatchKMeans
import datetime
from datetime import timedelta

from sklearn.metrics import silhouette_samples, silhouette_score


In [1]:
# total=pd.read_json("/content/drive/MyDrive/NLP_News/Data/total.json")
# total.columns = ['id', 'date','text','username','NE']
# total["ind"]=np.arange(total.shape[0])
# total.head()

In [2]:
# m_list=pd.read_csv("/content/drive/MyDrive/NLP_News/Data/till70000.json")
# m_list

In [ ]:
from datetime import datetime
from datetime import timedelta


#TODO reading data -> km models, test_data, total

for i in range(km_models.shape[0]):
  km_models.middle_time[i]=datetime.strptime(km_models.middle_time[i],'%Y-%m-%d %H:%M:%S')

for i in range(fakedata.shape[0]):
  fakedata.date[i]=datetime.strptime(fakedata.date[i], "%Y-%m-%d %H:%M:%S")
for i in range(truedata.shape[0]):
  truedata.date[i]=datetime.strptime(truedata.date[i], "%Y-%m-%d")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
# col_list = ['title','date','NE']
# truedata=pd.read_csv("/content/drive/MyDrive/NLP_News/Data/True_P.csv",skipinitialspace=True,usecols=col_list)

In [3]:
# truedata.columns = ['text','date','NE']
# truedata

In [ ]:
# col_list = ['title','date','NE']
# fakedata=pd.read_csv("/content/drive/MyDrive/NLP_News/Data/Fake_P.csv",skipinitialspace=True,usecols=col_list)

In [4]:
# fakedata.columns = ['text','date','NE']
# fakedata.head()

In [ ]:
def check_similarity(test_data,total_data,sim_rate):
  cv = TfidfVectorizer(stop_words="english")
  docs_total=total_data["text"].tolist()
  docs_test=test_data["text"].tolist()
  cv.fit(docs_total+docs_test)

  total_tweets=cv.transform(docs_total)
  test_tweets=cv.transform(docs_test)
  
  cosine_similarities_temp = np.dot(test_tweets[0],total_tweets.T).T
  cosine_similarities = np.reshape(cosine_similarities_temp.toarray(),(-1,))
  # print(cosine_similarities)
  most_similar=np.where(cosine_similarities>sim_rate)
  list_sim=[]
  for i in list(most_similar[0]):
    list_sim.append(int(total_data.ind[i:i+1]))
  return list_sim

In [ ]:
def find_week(sample_time,total_time):
  x=[]
  for i in range(len(total_time)):
    x.append(np.abs((total_time[i]-sample_time).days))
  return np.argmin(np.array(x))

In [ ]:
def sim_index_finder(test_data, models, total_data, sim_rate):
    sim_ind=[]
    all_ind=[]
    for i in range(test_data.shape[0]):

        print(i)
        test_data_date=test_data.date[i]
        model_date=models.middle_time
        ind=find_week(test_data_date,np.array(model_date))

        model_idf=pickle.loads(eval(models.tf_idf[ind]))
        model_km=pickle.loads(eval(models.model[ind]))

        true_tweets=model_idf.transform([test_data.text[i]])
        true_label=model_km.predict(true_tweets)
  
        total_tweets=model_idf.transform( total_data.text[ models.start_ind[ind] : models.end_ind[ind] ])
        total_label=np.array(model_km.predict(total_tweets))

        mask=total_label==true_label[0]
        d_total=total_data[ models.start_ind[ind] : models.end_ind[ind]]
        d_total=d_total[mask]
        all_ind.append(d_total.index.tolist())
        if d_total.shape[0]>0:

            sim_list = check_similarity(test_data[i:i+1],d_total,sim_rate)

            if len(sim_list)>0:
                sim_ind.append(sim_list)
                # print("kk")
            else:
                # print("ooo")
                sim_ind.append(-1)
        else:
            print("Error")
            sim_ind.append(-1)

    test_data_copy = test_data.copy()
    test_data_copy['similar_ind'] = sim_ind
    test_data_copy['all_ind'] = all_ind

    test_data_copy = test_data_copy[test_data_copy.similar_ind!=-1]
    test_data_copy.reset_index(drop=True, inplace=True)
    

    return test_data_copy


In [ ]:
similarity_rate = 0.31
true_dataset = sim_index_finder(true_data, km_models, total_data, similarity_rate)
fake_dataset = sim_index_finder(fake_data, km_models, total_data, similarity_rate)

true_dataset.to_json('../Data/Clustering/true_data.json')
fake_dataset.to_json('../Data/Clustering/fake_data.json')


#TODO writing files
